conda create -n antibody_prediction_safe python=3.10 -y
conda activate antibody_prediction_safe

# Core packages
conda install pytorch=2.0.1 torchvision torchaudio cpuonly -c pytorch -c nvidia

# Install compatible versions
pip install pytorch_lightning==2.0.7 biopython pandas tqdm scipy plotly seaborn matplotlib torchmetrics==0.11.4 transformers==4.29.2
pip install numpy==1.24.4
conda install conda-forge::pdbfixer
conda install bioconda::anarci
conda install -c conda-forge openmm==7.7.0
conda install -c bioconda abnumber
pip install igfold
pip install ImmuneBuilder
pip install abnumber
pip install freesasa




https://github.com/Graylab/IgFold
https://github.com/oxpig/ImmuneBuilder

In [2]:
# STEP 1A: Predict with IgFold and ImmuneBuilder

import os
import pandas as pd
from tqdm import tqdm
from igfold import IgFoldRunner
from ImmuneBuilder import ABodyBuilder2

# --- Setup ---
output_base = "/home/eva/Structure_model_benchmark/predictions_output"
models = ["igfold", "immunebuilder"]
repeats = ["1", "2", "3"]

# Create folders
for model_name in models:
    for rep in repeats:
        os.makedirs(os.path.join(output_base, f"{model_name}_{rep}"), exist_ok=True)

# --- Load Input Sequences ---
csv_path = "/home/eva/Structure_model_benchmark/antibody.csv"
df = pd.read_csv(csv_path)

# --- Load Models ---
igfold_runner = IgFoldRunner()
immunebuilder_runner = ABodyBuilder2()

# --- Prediction Functions ---
def predict_igfold(name, vh, vl, out_path):
    igfold_runner.fold(pdb_file=out_path, sequences={"H": vh, "L": vl}, do_refine=False, do_renum=False, use_openmm=True)

def predict_immunebuilder(name, vh, vl, out_path):
    ab = immunebuilder_runner.predict({"H": vh, "L": vl})
    ab.save(out_path)

model_functions = {
    "igfold": predict_igfold,
    "immunebuilder": predict_immunebuilder
}

# --- Prediction Loop ---
for idx, row in tqdm(df.iterrows(), total=len(df)):
    name = row['name']
    vh_seq = row['vh']
    vl_seq = row['vl']

    for model_name in models:
        for rep in repeats:
            out_path = os.path.join(output_base, f"{model_name}_{rep}", f"{name}_{model_name}_{rep}.pdb")
            try:
                model_functions[model_name](name, vh_seq, vl_seq, out_path)
            except Exception as e:
                print(f" Error predicting {name} ({model_name} rep {rep}): {e}")

print(" IgFold + ImmuneBuilder prediction (3x repeats) complete.")



    The code, data, and weights for this work are made available for non-commercial use 
    (including at commercial entities) under the terms of the JHU Academic Software License 
    Agreement. For commercial inquiries, please contact awichma2[at]jhu.edu.
    License: https://github.com/Graylab/IgFold/blob/main/LICENSE.md
    
Loading 4 IgFold models...
Using device: cpu
Loading /home/eva/miniconda3/envs/antibody_prediction_safe/lib/python3.10/site-packages/igfold/trained_models/IgFold/igfold_1.ckpt...
Loading /home/eva/miniconda3/envs/antibody_prediction_safe/lib/python3.10/site-packages/igfold/trained_models/IgFold/igfold_2.ckpt...
Loading /home/eva/miniconda3/envs/antibody_prediction_safe/lib/python3.10/site-packages/igfold/trained_models/IgFold/igfold_3.ckpt...
Loading /home/eva/miniconda3/envs/antibody_prediction_safe/lib/python3.10/site-packages/igfold/trained_models/IgFold/igfold_5.ckpt...
Successfully loaded 4 IgFold models.
Loaded AntiBERTy model.


  0%|          | 0/121 [00:00<?, ?it/s]

Completed folding in 14.28 seconds.
Completed folding in 15.05 seconds.
Completed folding in 13.98 seconds.


  1%|          | 1/121 [02:52<5:45:55, 172.97s/it]

Completed folding in 14.46 seconds.
Completed folding in 13.88 seconds.
Completed folding in 14.19 seconds.


...

Completed folding in 15.43 seconds.
Completed folding in 15.60 seconds.
Completed folding in 14.36 seconds.


  2%|▏         | 3/121 [07:50<5:05:52, 155.53s/it]

Completed folding in 14.54 seconds.


  2%|▏         | 3/121 [08:13<5:23:38, 164.56s/it]


KeyboardInterrupt: 